In [54]:
path_ = 'C:/Users/Marie/Desktop/MSc DSBA/1. Big Data Analytics/1. Project/Full Data/Cleaned Data/'

### Importing packages

In [60]:
from os import listdir
import pandas as pd
import numpy as np

In [61]:
print(listdir(path_))

['full_daily_data.csv', 'full_data.csv', 'full_hourly_data.csv', 'holidays.csv', 'hourly_full_data.csv', 'log.txt', 'log_year.txt', 'stations.csv', 'Weather Data']


### Loading data

In [62]:
df = pd.read_csv(path_ + 'full_data.csv').drop(['cnt', 'casual', 'registered', 'wkday'], axis = 1)
df_hour = pd.read_csv(path_ + 'full_hourly_data.csv')
df = pd.merge(left = df, right = df_hour, on =  ['year', 'month', 'day'], how = 'outer')

### Cosine distance feature engineering

In [63]:
df['hr_sin'] = np.sin(df.hour*(2.*np.pi/24))
df['hr_cos'] = np.cos(df.hour*(2.*np.pi/24))
df['wkday_sin'] = np.sin(df.weekday*(2.*np.pi/7))
df['wkday_cos'] = np.cos(df.weekday*(2.*np.pi/7))
df['sunrise_sin'] = np.sin(df.sunrise*(2.*np.pi/24))
df['sunrise_cos'] = np.cos(df.sunrise*(2.*np.pi/24))
df['sunset_sin'] = np.sin(df.sunset*(2.*np.pi/24))
df['sunset_cos'] = np.cos(df.sunset*(2.*np.pi/24))
df['mnth_sin'] = np.sin((df.month-1)*(2.*np.pi/12))
df['mnth_cos'] = np.cos((df.month-1)*(2.*np.pi/12))
df['light'] = np.logical_and(df.hour>= df.sunrise, df.hour<df.sunset+1)

### Setting index

In [64]:
df = df.set_index('index', drop = True)
# df_targets = df[['casual', 'registered', 'cnt']]
# df = df.drop(['casual', 'registered', 'cnt'], axis = 1)

In [66]:
meteo_col = ['max_temp', 'min_temp', 'wind_speed', 'wind_temp', 'rain', 'hum',
       'visibility', 'clouds', 'chaleur_index', 'rosee', 'pressure', 'sunrise',
       'sunset', 'light']
time_col = ['holiday', 'weekday', 'workday', 'avg_temp', 'string_date',
       'year', 'month', 'day', 'atemp', 'wkday', 'hour', 'hr_sin', 'hr_cos', 'wkday_sin', 'wkday_cos',
       'sunrise_sin', 'sunrise_cos', 'sunset_sin', 'sunset_cos', 'mnth_sin',
       'mnth_cos']
target_col = ['casual',
       'registered', 'cnt']

In [11]:
df = df[pd.notnull(df.atemp)]
df = df.sort_values(['year', 'month', 'day', 'hour'])
df = df.fillna(method = 'ffill')
df = df.drop(["rosee", "chaleur_index", 'sunrise', 'sunset', 'hour'], axis = 1)

In [12]:
for col in df.columns:
    print(col + '\t', sum(pd.isnull(df[col])))

max_temp	 0
min_temp	 0
wind_speed	 0
wind_temp	 0
rain	 0
hum	 0
visibility	 0
clouds	 0
pressure	 0
holiday	 0
weekday	 0
workday	 0
avg_temp	 0
string_date	 0
year	 0
month	 0
day	 0
atemp	 0
wkday	 0
hr_sin	 0
hr_cos	 0
wkday_sin	 0
wkday_cos	 0
sunrise_sin	 0
sunrise_cos	 0
sunset_sin	 0
sunset_cos	 0
mnth_sin	 0
mnth_cos	 0
light	 0


In [13]:
df_targets = df_targets.sort_index()
df_targets = df_targets.iloc[:-sum(pd.isnull(df_targets.index)),:]

In [14]:
df = df.drop(['wkday', 'weekday', 'day', 'month', 'string_date'], axis = 1)

In [15]:
col_reg = [col for col in df.columns if 'cos' not in col]
col_reg = [col for col in col_reg if 'sin' not in col]

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
scaler = MinMaxScaler()
df[col_reg] = scaler.fit_transform(df[col_reg])

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [19]:
### Predicting cnt
y = df_targets['cnt']
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = .4)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size = .2)

In [38]:
lr = LinearRegression()
rf = GradientBoostingRegressor(n_estimators=1000)

In [39]:
lr_fit = lr.fit(X_train, y_train)

In [40]:
rf_fit = rf.fit(X_train, y_train)

In [41]:
y_pred_cv = rf_fit.predict(X_cv)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_cv, y_pred_cv))

44137.4478133


In [42]:
y_pred_cv = pd.DataFrame({'predictions': y_pred_cv, 'actual_values':y_cv})

In [43]:
y_pred_cv.index = X_cv.index

In [44]:
y_pred_cv.sort_index()

,actual_values,predictions
index,,
2010-09-17,6,188.298661
2010-09-17,2,149.851563
2010-09-19,5,-29.717051
2010-09-20,18,57.211456
2010-09-20,11,151.423625
2010-09-20,19,24.720715
2010-09-20,13,2.866578
2010-09-21,16,-30.537191
2010-09-21,20,23.175010
